# Transport periodique front-tracking en IJK

## Introduction
 
 Validation made by: G.B.
 
 Report generated 26/03/2025

### Description
 




### TRUST parameters
 
* Version TRUST :  


In [ ]:
# Prepare to do here later

In [ ]:
from trustutils import run
 
run.reset() 
run.initBuildDirectory()
run.executeCommand("./prepare")
dic_cas = {"procs":"1 1 1", "freprise_interf": "init.lata", "mode":"WRITE_PASS", "Debogfile":"seq.txt", "Debogseuil":"-1.", \
           "vy":"0.2", "fsauv":"ConvMultiSpherePerio.repr", "frepr":"ConvMultiSpherePerio.repr", \
           "lata_meshname":"FTMESH" }
run.addCaseFromTemplate("ijkft_ConvectionMultiSphere_template.data",".",dic_cas, nbProcs=1,targetData="ijkft_ConvectionMultiSphere_seq.data")

dic_cas = {"procs":"2 2 2", "freprise_interf": "init.lata", "mode":"CHECK_PASS", "Debogfile":"seq.txt", "Debogseuil":"0.", \
           "vy":"0.2", "fsauv":"ConvMultiSpherePerio_par8.repr", "frepr":"ConvMultiSpherePerio_par8.repr", \
           "lata_meshname":"FTMESH" }
run.addCaseFromTemplate("ijkft_ConvectionMultiSphere_template.data",".",dic_cas, nbProcs=8,targetData="ijkft_ConvectionMultiSphere_par8.data")

# Remove Debog from template (Debog and 6 following lines) for restart and remove nom_sauvegarde
run.executeCommand("sed -e '/DebogIJK/,+6d' -e '/nom_sauvegarde/d' ijkft_ConvectionMultiSphere_template.data > ijkft_ConvectionMultiSphere_template2.data")

dic_cas = {"procs":"1 1 1", "freprise_interf": "ConvMultiSpherePerio.repr.lata", \
           "vy":"-0.25*sin(x*6.28318530717959/0.4)*sin(z*6.28318530717959/0.5)", \
           "frepr":"ConvMultiSpherePerio.repr", \
           "lata_meshname":"INTERFACES" }
run.addCaseFromTemplate("ijkft_ConvectionMultiSphere_template2.data",".",dic_cas, nbProcs=1,targetData="ijkft_ConvectionMultiSphere_seq_reprise.data")

dic_cas = {"procs":"2 2 2", "freprise_interf": "ConvMultiSpherePerio_par8.repr.lata",  \
           "vy":"-0.25*sin(x*6.28318530717959/0.4)*sin(z*6.28318530717959/0.5)", \
           "frepr":"ConvMultiSpherePerio_par8.repr", \
           "lata_meshname":"INTERFACES" }
run.addCaseFromTemplate("ijkft_ConvectionMultiSphere_template2.data",".",dic_cas, nbProcs=8,targetData="ijkft_ConvectionMultiSphere_par8_reprise.data")

run.executeCommand("sed -i \"/nom_reprise/d\" ijkft_ConvectionMultiSphere_seq.data")
run.executeCommand("sed -i \"/nom_reprise/d\" ijkft_ConvectionMultiSphere_par8.data")

run.printCases()
run.runCases(preventConcurrent=True)

## Maillage genere par cree\_multi\_bulles.sh
 


### Etat initial genere par cree\_multi\_bulles.sh
 
 

In [ ]:
from trustutils import visit
 
visu = visit.Show("init.lata","Mesh","FTMESH")
visu.addField("init.lata","Pseudocolor","COMPO_CONNEXE_ELEM_FTMESH")
visu.plot()

## Verification of sequential and parallel calculations (comparing initial / final states)

In [ ]:
from trustutils import visit
def myFigures(sub_calc="", pretitle="Initial "):
    inst ={0:"initial", -1:"final"}
    # A first dummy plot because empty option fails. 
    visu = visit.Show("ijkft_ConvectionMultiSphere_seq.lata","Mesh",name="DOM", plotmesh=False, \
                          nX=2, nY=2,size=12,title=f"{pretitle} runs (seq or par8)")
    for j,calc in enumerate(["seq", "par8"]):
        part=f"{calc}{sub_calc}"
        basename=f"ijkft_ConvectionMultiSphere_{part}.lata"
        for i, k in enumerate(inst.keys()):
            visu.add(basename,"Pseudocolor","VISCOSITY_ELEM_DOM", \
                          xIndice=i, yIndice=j,plotmesh=True, iteration=k, title=f"{calc.upper()} -- {inst[k].upper()} STEP")
            visu.visitCommand("PseudocolorAtts = PseudocolorAttributes()")
            visu.visitCommand("PseudocolorAtts.opacityType = PseudocolorAtts.Constant ")
            visu.visitCommand("PseudocolorAtts.opacity = 0.1")
            visu.visitCommand("SetPlotOptions(PseudocolorAtts)")
            visu.addField(basename,"Pseudocolor","COMPO_CONNEXE_ELEM_INTERFACES",plotmesh=True)
            visu.addField(basename,"Mesh","DOM_EXT")
            pass
        pass
    visu.plot()
    return

myFigures()


## Restart cases (seq/par8) 

In [ ]:
myFigures("_reprise", "Restart ")

### Coupe du champ de vitesse Y sur le domaine etendu
 
 

In [ ]:
def myFigures2(slice=False, interf_var="COMPO_CONNEXE_ELEM_INTERFACES"):
    runs ={"initial":"", "reprise":"_reprise"}
    # A first dummy plot because empty option fails. 
    visu = visit.Show("ijkft_ConvectionMultiSphere_seq.lata","Mesh",name="DOM", plotmesh=False, \
                          nX=2, nY=2,size=12,title=f"Runs (seq or par8) (precalc or restart) (last step)")
    for j,calc in enumerate(["seq", "par8"]):
        for i, k in enumerate(runs.keys()):
            part=f"{calc}{runs[k]}"
            basename=f"ijkft_ConvectionMultiSphere_{part}.lata"
            print(basename)
            visu.add(basename,"Pseudocolor","VELOCITY_FT_Y_FACES_DOM_EXT_dual", \
                          xIndice=i, yIndice=j,plotmesh=True, iteration=-1, title=f"{calc.upper()} -- {k.upper()}")
            visu.visitCommand("PseudocolorAtts = PseudocolorAttributes()")
            visu.visitCommand("PseudocolorAtts.opacityType = PseudocolorAtts.Constant ")
            visu.visitCommand("PseudocolorAtts.opacity = 0.1")
            visu.visitCommand("SetPlotOptions(PseudocolorAtts)")
            visu.addField(basename,"Pseudocolor",interf_var,plotmesh=False)
            if slice:
                visu.slice(origin=[0,0,0.1],var='z',all=1, type_op="slice2d")
            pass
        pass
    visu.plot()
    return

myFigures2(slice=True)

### Decoupage parallele des interfaces
 
 

In [ ]:
myFigures2(slice=False, interf_var="FACETTE_PE_OWNER_ELEM_INTERFACES")